# MODEL VERIFICATION

When creating the model from scratch and manipulating the data in different ways to adapt it to the use case, verifying the model at each step turns out to be important.

During LAB2 notebook, we already verified the FINN-ONNX model, which indicates that we did the job right on our side, but what if something goes wrong in the way we apply further transformations ?

To make sure we are ready for FPGA inference, verification is a very important step to avoid hours of useless hardware dubugging.

Verifications convered by this notebook :

- HLS layers verification using C++
- RTL output verification using PyVerilator

This notebook was based on [this example](https://github.com/Xilinx/finn/blob/main/notebooks/end2end_example/bnn-pynq/tfc_end2end_verification.ipynb) from FINN tutorials.

As you will see, verification will be fairly easy as FINN provides a very user-friendly API for these tools.

# C++ Simulation

First, execute LAB2, we will grab the models from the common ```/tmp/finn_dev_yourusername/``` output folder

In [5]:
from qonnx.core.modelwrapper import ModelWrapper
model_cppsim = ModelWrapper("/tmp/finn_dev_rootmin/to_hw_conv.onnx")

We then define the "golden reference" for comparison

In [ ]:
import numpy as np
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as oxe

input_tensor = input_a = np.random.uniform(low=0, high=255, size=(28*28)).astype(np.uint8).astype(np.float32)
input_dict = {"global_in": input_tensor.reshape(1,28*28)}
golden_model = ModelWrapper("/tmp/finn_dev_rootmin/full_preproc.onnx")
output_dict = oxe.execute_onnx(golden_model, input_dict)
golden_output = output_dict[list(output_dict.keys())[0]]

print(golden_output)

We will generate the different source code : ```PrepareCppSim``` and executables : ```CompileCppSim```

In [ ]:
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from qonnx.transformation.general import GiveUniqueNodeNames

model_cppsim = model_cppsim.transform(GiveUniqueNodeNames())
model_cppsim = model_cppsim.transform(PrepareCppSim())
model_cppsim = model_cppsim.transform(CompileCppSim())

from finn.util.visualization import showSrc, showInNetron

model_cppsim.save("/tmp/finn_dev_rootmin/cppsim.onnx")
showInNetron("/tmp/finn_dev_rootmin/cppsim.onnx")


graph manipulation reminder : [cutomOp Docs](https://finn.readthedocs.io/en/latest/source_code/finn.custom_op.html#module-qonnx.custom_op.registry)

In [ ]:
# Look at the generated files
from qonnx.custom_op.registry import getCustomOp

model = ModelWrapper("/tmp/finn_dev_rootmin/cppsim.onnx")

fc0 = model.graph.node[0]
fc0w = getCustomOp(fc0)
cpp_code_dir = fc0w.get_nodeattr("code_gen_dir_cppsim")

!ls {cpp_code_dir}

## Simulate execution

In [13]:
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode

model_cppsim = model_cppsim.transform(SetExecMode("cppsim"))
model_cppsim.save("/tmp/finn_dev_rootmin/cppsim_exec.onnx")

In [ ]:
import numpy as np
import onnx.numpy_helper as nph
import qonnx.core.onnx_exec as oxe

input_dict = {"global_in": input_tensor.reshape(1,28*28)}

parent_model = ModelWrapper("/tmp/finn_dev_rootmin/df_part.onnx")
sdp_node = parent_model.graph.node[0]
child_model = "/tmp/finn_dev_rootmin/cppsim_exec.onnx"
getCustomOp(sdp_node).set_nodeattr("model", child_model)
output_dict = oxe.execute_onnx(parent_model, input_dict)
output_cppsim = output_dict[list(output_dict.keys())[0]]

try:
    print(golden_output, output_cppsim)
    assert np.isclose(output_cppsim, np.where(golden_output==np.amax(golden_output)), atol=1e-3).all()
    print("Results are the same!")
except AssertionError:
    assert False, "The results are not the same!"

Great results are the same ! Note that this very small exmaple was done as an example and compares simple top label output. You can use the exmaple in a loop to check for hundreds of random sample et even setup a dataloader and testing loop for verification like we did like in LAB2.